In [1]:
# установка внешних программ и библиотек на виртуальную машину
!git clone --recurse-submodules https://github.com/KarlsruheMIS/KaMIS.git
%cd KaMIS
!./compile_withcmake.sh

%cd ..


!pip3 install python-sat
#!sudo apt install minisat
!git clone https://github.com/arminbiere/kissat.git
%cd kissat
!./configure
!make

%cd ..

!cp KaMIS/deploy/redumis redumis
!cp KaMIS/deploy/online_mis online_mis
!cp kissat/build/kissat kissat1



Cloning into 'KaMIS'...
remote: Enumerating objects: 1481, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 1481 (delta 39), reused 45 (delta 27), pack-reused 1399
Receiving objects: 100% (1481/1481), 12.78 MiB | 12.74 MiB/s, done.
Resolving deltas: 100% (830/830), done.
/content/KaMIS
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test COMPILER_SUPPORTS_FUNROLL_LOOPS
-- Performing Test COMPILER_SUPPORTS_FUNROLL_LOOPS - Success
-- Performing Test COMPILER_SUPP

In [1]:
# функции из библиотеки
# раскраска графа в формате  DIMACS, преобразование в CNF

import math
import numpy as np
from numpy import linalg as LA
import networkx as nx

#import matplotlib.pylab as plt

#from google.colab import files 
#files.upload() 
#files.download("file.txt")

#https://pysathq.github.io/
#!pip install python-sat
#import pysat
from pysat.formula import CNF
from pysat.solvers import *

import os
import sys
import random
from zipfile import ZipFile
#from tqdm import tqdm_notebook as tqdm
from itertools import combinations, permutations
import glob

out_dir = ""

class Utils:
    
    def read_dimacs_graph(file = 'graph.col'):
        
        if not (os.path.exists(file) and os.path.getsize(file) > 0):        
            raise Exception("File " + file + " not found")
        
        nodes = []    
        edges = []
        labels = []
        got_labels = False
        nnodes = nedges = 0
        
        with open(file, 'r') as f:
            for line in f:
                line = [l.strip() for l in line.split(' ')]
                if line[0] == 'c': #comment
                    continue
                elif line[0] == 'p': #problem
                    nnodes = int(line[2])
                    nedges = int(line[3])
                    nodes = list(range(1, nnodes + 1))
                    labels = [0] * nnodes
                elif line[0] == 'e': #edge
                    edges.append((int(line[1]), int(line[2])))
                elif line[0] == 'l':
                    labels[int(line[1]) - 1] = int(line[2])
                    got_labels = True

        if got_labels:        
            nodes = [(n, {'c' : l}) for n, l in zip(nodes, labels)]

        g = nx.Graph()
        g.add_nodes_from(nodes)
        g.add_edges_from(edges)
        return g

    def write_dimacs_graph(file = 'graph.col', g = nx.Graph(), comments = []):
        with open(file, 'w') as f:
            for c in comments:
                f.write("c " + c + "\n")
            f.write("p EDGE {} {}\n".format(g.number_of_nodes(), g.number_of_edges()))
            for u, v in g.edges():
                f.write("e {} {}\n".format(u, v))
            for node in g.nodes():
                if 'c' in g.node[node]:
                    f.write("l {} {}\n".format(node, g.node[node]['c']))

    def draw_with_colors(g = nx.Graph()):
        color_map = []
        for node in g.nodes():
            if 'c' in g.node[node]:
                color_map.append(g.node[node]['c'] * 10)            
        nx.draw(g, pos = nx.spring_layout(g, scale=2), node_color=color_map, with_labels=True, cmap = plt.cm.jet)

    def write_proof(file = "proof.txt", proof = []):
        with open(file, 'w') as f:
            for p in proof:
                f.write("%s\n" % str(p))

    def zip_files(file = "archive.zip", files = []):
        with ZipFile(file, 'w') as archive:
            for f in set(files):                
                if not (os.path.exists(file) and os.path.getsize(file) > 0):        
                    raise Exception("File " + file + " not found")
                archive.write(f)
                
def find_triangle(g = nx.Graph()):
    for a in g:
        for b, c in combinations(g[a], 2):
            if b in g[c]:
                return [a, b, c]
    return []
             
    #return set(frozenset([a, b, c]) for a in g for b, c in combinations(g[a], 2) if b in g[c])


def find_isolates(g = nx.Graph()):
    isolates = []
    for n in g:
        if g.degree(n) == 0:
            isolates.append(n)
    return isolates

class ColMap:
    
    def __init__(self, g = nx.Graph(), ncolors = 40):
        
        self.ncolors = ncolors
        self.cmap = dict()
        self.cunmap = dict()
    
        i = 1
        for node in g.nodes():
            for color in range(1, ncolors + 1):            
                self.cmap[(node, color)] = i
                self.cunmap[i] = (node, color)
                i += 1    

    def enc(self, node, color):
        return self.cmap[(node, color)]

    def dec(self, node_color):
        return self.cunmap[node_color]

class ColSAT:
   
    def __init__(self, g = nx.Graph(), ncolors = 10):
        
        self.ncolors = ncolors
        self.g = g.copy()
        self.cmap = ColMap(g, ncolors)        

    def check_coloring(self):
        for n1, n2 in self.g.edges():
            if 'c' not in self.g.node[n1] or 'c' not in self.g.node[n2]:
                return False
            if self.g.node[n1]['c'] == self.g.node[n2]['c']:
                return False
        return True
    
    def apply_model(self):
        
        check = set()
        for var in self.model[self.model > 0]:        
            node, color = self.cmap.dec(var)
            self.g.node[node]['c'] = color
            if (node, color) in check:
                raise Exception("Two colors for one node???")
            else:
                check.add((node, color))
        
        self.colored = self.check_coloring()
        
        if self.colored != self.solved:
            raise Exception("Something went wrong!")
        
        return self.colored
        
    def build_cnf(self, ):
        
        self.formula = CNF()
        colors = list(range(1, self.ncolors + 1))    

        maxc = -1        # condition k<12 is not for general case
        k = 0
        for clique in nx.find_cliques(self.g):
            if len(clique)>maxc:
                maxclique = clique.copy()  
                maxc = len(clique)
            if k<12:
                k += 1
                continue
           
            col = 1
            if len(maxclique)>self.ncolors:
                print("Size of the maximal clique is {}".format(len(maxclique)))
                raise Exception("Size of the maximal clique is {} > {}".format(len(maxclique),ncolors))
            for v in maxclique:
                self.formula.append([self.cmap.enc(v, col)])
                col += 1
            break

        for n1, n2 in self.g.edges():
            for c in colors:            
                self.formula.append([-self.cmap.enc(n1, c), -self.cmap.enc(n2, c)])


        for n in self.g.nodes():
            #if not n in specials:
            self.formula.append([self.cmap.enc(n, c) for c in colors])
            #for c1 in colors:
            #    for c2 in colors:
            #        if c1 < c2:
            #            self.formula.append([-self.cmap.enc(n, c1), -self.cmap.enc(n, c2)])
        
        return self.formula
    
    def solve_cnf(self, solver = ''):
        
        triangle = find_triangle(self.g)
        assumptions = []
        if len(triangle) > 0:            
            assumptions = [self.cmap.enc(triangle[0], 1), self.cmap.enc(triangle[1], 2), self.cmap.enc(triangle[2], 3)]
            
        #Glucose3, Glucose4, Lingeling, MapleChrono, MapleCM, Maplesat, Minisat22, MinisatGH
        #with Glucose4(bootstrap_with=self.formula.clauses, with_proof=True) as ms:        
        with Lingeling(bootstrap_with=self.formula.clauses) as ms:
            self.solved = ms.solve(assumptions=assumptions)
            if self.solved:
                self.model = np.array(ms.get_model())
                self.apply_model()
            else:                
                self.proof = []#ms.get_proof()
                self.colored = False
                
        return self.solved






In [2]:
# выбор графа на вершинах гиперкуба (покрышки), удаление независимого множества,
# проверка существования раскраски

import networkx as nx

def dist(a,b):
  return sum([int(c) for c in bin(a^b).split('b')[1]])

def bin_graph(n, k, trim = []):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    for t in trim:
      if dist(t,i)>k:
        flag = False
        break
    if flag:
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def small_graph(n, k, dists = [2,4,8]):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i) not in dists:
      continue
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def bin_graph1(n, k, trim = []):
  g = nx.Graph()
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    if dist(0,i)>k:
      flag = False
      
    if flag or i in set(trim):
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)


def r_graph(n, k):      # граф запретов,
  g = nx.Graph()        # ребрами соединены вершины на расстоянии >k
  for i in range(0,2**n):
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)>k):
        g.add_edge(i,j)
  return g

def write_metis(g0,fname):    # запись графа во входном формате KaMIS
  g = nx.convert_node_labels_to_integers(g0)
  with open(fname,"w") as f:
    f.write("{} {}\n".format(g.number_of_nodes(),g.number_of_edges()))
    for v in range(g.number_of_nodes()):
      f.write(" ".join([str(i+1) for i in sorted(list(g[v]))])+"\n")    

def write_cnf(g,ncolors,filename):  # запись КНФ для SAT-решателя    
  problem = ColSAT(g, ncolors)
  problem.build_cnf().to_file(filename)

def select_mis(g,filename):  # возвращает независимое множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==0:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      


def delete_mis(g,filename):  # удаляет независимосе множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==1:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      

# размерность, расстояние, номера вершин, по окрестностям которых 
# выполняется отсечение
 
r = bin_graph(10,6,trim=[0, 984, 730, 826, 1000, 718, 857])


#g = bin_graph(9,6,trim=[0,63])  
#print(r.number_of_nodes())
#print(r.number_of_edges())
#write_metis(r,"test.graph")

#!./redumis test.graph --output test.mis --time_limit=30

#g = r.copy()
g1 = r.copy()

print(g1.number_of_nodes())
print(g1.number_of_edges())

#write_cnf(g1,10,"test.cnf")

#!./kissat1 test.cnf

nmis = 0

283
15247


In [3]:
FILENAME = "test_11"
COLORSS = 0

def is_not_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return True

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if any(["UNSATISFIABLE" in line for line in outputt]):
         return True
    return False

def is_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return False

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if all(["UNSATISFIABLE" not in line for line in outputt]):
        if any(["SATISFIABLE" in line for line in outputt]):
            return True
    return False

def check_colorable_fast(graph, max_colors=100, timer=120):
    if graph.number_of_edges() == 0:
        return (None, 0)

    max_uncolorable = None
    min_colorable = None
    
    left_border = 1
    right_border = max_colors + 1
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_not_colorable(graph, color_count_to_check, timer):
            left_border = color_count_to_check
        else:
            right_border = color_count_to_check
    max_uncolorable = left_border

    left_border = 1
    right_border = max_colors
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_colorable(graph, color_count_to_check, timer):
            right_border = color_count_to_check
        else:
            left_border = color_count_to_check
    if right_border == max_colors:
        if is_colorable(graph, max_colors, timer):
            min_colorable = right_border 
        return (max_uncolorable, min_colorable)
    else:
        min_colorable = right_border
        return (max_uncolorable, min_colorable)
    return (None, None)

# my_graph = small_graph(10, 6)
# qq = my_graph.copy()
# check_colorable_fast(qq)

# Ниже рабочая зона (Да)#


In [4]:
FILENAME = "NoK4"

In [5]:
# !!! this is late-stage thing
base_conf = [0, 945, 910]

k2s = [(0, 945),
       (0, 910),
       (945, 910)]
kl3s = [(0, 910, 945)]

k_2 = []
k_4 = []
k_6 = []

for i in range(2**10):
    if dist(0, i) % 2 == 1:
        continue
    flag = True
    for base_v in base_conf:
        if dist(i, base_v) == 0 or dist(i, base_v) > 6:
            flag = False
            continue
    big_flag = True
    for kl3 in kl3s:
        fflag = False
        for v in kl3:
            if dist(i, v) < 6:
                fflag = True
                break
        if not fflag:
            big_flag = False
            break
    if not big_flag:
        continue
    if flag:
        if dist(0, i) == 2:
            k_2.append(i)
        if dist(0, i) == 4:
            k_4.append(i)
        if dist(0, i) == 6:
            k_6.append(i)

print(len(k_6), len(k_4), len(k_2))

150 160 33


In [6]:
BIG_NUM = 3
COUNTER = 0
SAVED = []
BPM = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

def simple_cook_graph_no_K4(base_conf, add_conf, kl3s, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
    if COUNTER % 50 == 0:
        print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue

        big_flag = True
        for kl3 in kl3s:
            fflag = False
            for v in kl3:
                if dist(i, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if big_flag:
            g.add_node(i)
    for i in list(g.nodes()):
        for j in g.nodes():
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

def sort_to_check(to_check, base_conf, add_conf):
    to_to = [[a, 0] for a in to_check]
    for a in to_to:
        for b in base_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
        for b in  add_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
    tbd = sorted(to_to, key=lambda x: (-x[1], x[0]))
    return [a[0] for a in tbd]


def bruteforce_nolimits_nok4(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM, SAVED
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl3 in kl3s:
            fflag = False
            for v in kl3:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        if how_much > 1:
            bruteforce_nolimits_nok4(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_no_K4(base_conf=base_conf, add_conf=add_conf + [elem], kl3s = kl3s + kl3s_updated, k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                SAVED.append((base_conf, add_conf + [elem]))
                print(base_conf, add_conf + [elem])
    if how_much == BIG_NUM:
        print("analysing {} finished".format(BIG_NUM))

def bruteforce_nolimits_nok4_short(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], how_much=4, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM, SAVED
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl3 in kl3s:
            fflag = False
            for v in kl3:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))

        if how_much == 1:
            grph = simple_cook_graph_no_K4(base_conf=base_conf, add_conf=add_conf + [elem], kl3s = kl3s + kl3s_updated, k_2=k_2, k_4=k_4, k_6=[], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
                SAVED.append((base_conf, add_conf + [elem]))
        if how_much > 1:
            bruteforce_nolimits_nok4_short(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)

    if how_much == BIG_NUM:
        print("analysing {}-or-less-but-at-least-1 finished".format(BIG_NUM))

In [7]:
kv = base_conf

In [ ]:
# %%time
# BIG_NUM=5
# print("COLORED", COLORSS)
# print('to check:', len(k_6), len(k_6))
# bruteforce_nolimits_nok4(k_6, kv, [], k_2, k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
# print("AFTER ALL-COLORED", COLORSS)

In [ ]:
# %%time
# BIG_NUM=4
# print("COLORED", COLORSS)
# print('to check:', len(k_6), len(k_6))
# bruteforce_nolimits_nok4_short(k_6, kv, [], k_2, k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
# print("AFTER ALL-COLORED", COLORSS)

In [10]:
%%time
BIG_NUM=2
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6, kv, [], k_2, k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

COLORED 150
to check: 150 150
[[0, 0], [65, 149], [0, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [126, 149], [0, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [46, 148], [1, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [105, 148], [1, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [18, 147], [2, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [78, 147], [2, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [139, 147], [2, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [61, 146], [3, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [122, 146], [3, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [45, 145], [4, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [104, 145], [4, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [21, 144], [5, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [80, 144], [5, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 143], [6, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [66, 143], [6, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [127, 143], [6, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [53, 142], [7, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [114, 142], [7, 150], [0, 0], [0, 0], 

In [9]:
%%time
BIG_NUM=1
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6, kv, [], k_2, k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)


COLORED 0
to check: 150 150
[[0, 0], [49, 150], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [99, 150], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [149, 150], [0, 0], [0, 0], [0, 0], [0, 0]]
analysing 1-or-less-but-at-least-1 finished
AFTER ALL-COLORED 150
CPU times: user 1min 2s, sys: 3.1 s, total: 1min 5s
Wall time: 1min 30s


In [8]:
# на самом деле не симпл
def simple_cook_graph_no_K4(base_conf, add_conf, kl3s, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
    if COUNTER % 50 == 0:
        print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    good_vrtx = []
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue

        big_flag = True
        for kl3 in kl3s:
            fflag = False
            for v in kl3:
                if dist(i, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if big_flag:
            g.add_node(i)
            good_vrtx.append(i)

    prepared = []
    for i in bans:
        for j in bans:
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
                prepared.append((i, j))
    
    for i in bans:
        for j in good_vrtx:
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)

    for i in good_vrtx:
        for j in bans:
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)

    for i in good_vrtx:
        for j in good_vrtx:
            if (i<j) and (dist(i,j)==k):
                flag = True
                for prep in prepared:
                    if dist(i, prep[0]) == k and dist(i, prep[1]) == k and dist(j, prep[0]) == k and dist(j, prep[1]) == k:
                        flag = False
                if flag:
                    g.add_edge(i,j)
    

    # for i in list(g.nodes()):
    #     for j in g.nodes():
    #         if (i<j) and (dist(i,j)==k):
    #             g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

In [11]:
%%time
BIG_NUM=3
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4(k_6[:80], kv, [], k_2+k_6[80:], k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

COLORED 17
to check: 150 150
[[0, 0], [45, 78], [0, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [37, 77], [1, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [29, 75], [3, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [19, 72], [6, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [15, 71], [7, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [13, 70], [8, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [11, 69], [9, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [6, 68], [10, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [5, 66], [12, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [3, 64], [14, 79], [0, 80], [0, 0], [0, 0]]
[0, 945, 910] [159, 175, 189]
[[0, 0], [58, 64], [14, 79], [0, 80], [0, 0], [0, 0]]
[0, 945, 910] [159, 183, 189]
[0, 945, 910] [159, 183, 190]
[[0, 0], [46, 63], [15, 79], [0, 80], [0, 0], [0, 0]]
^C
[0, 945, 910] [159, 187, 717]



KeyboardInterrupt



In [9]:
print("omega lul")

omega lul


In [10]:
FILENAME = "NoK4_0_1_2_3_01"

In [ ]:
%%time
BIG_NUM=4
SAVED = []
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4(k_6[:80], kv, [], k_2+k_6[80:], k_4, k2s, kl3s, BIG_NUM, 40, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

COLORED 33
to check: 150 150
[[0, 0], [25, 76], [1, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [39, 75], [2, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [45, 73], [4, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [51, 72], [5, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [65, 71], [6, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [6, 68], [9, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [20, 65], [12, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [34, 64], [13, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [46, 63], [14, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [6, 59], [18, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [24, 58], [19, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [44, 55], [22, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [28, 51], [26, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [4, 49], [28, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [35, 47], [30, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [9, 44], [33, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [27, 43], [34, 78], [0, 79], [0, 80], [0, 0]]
[[0, 0], [8, 41], [36, 78], [0, 79], [0, 80], 

[[0, 0], [2, 7], [62, 70], [8, 79], [0, 80], [0, 0]]
[[0, 0], [57, 67], [1, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [14, 63], [5, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [19, 62], [6, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [47, 59], [9, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [30, 55], [13, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [48, 54], [14, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [19, 51], [17, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [38, 50], [18, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [22, 47], [21, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [32, 45], [23, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [5, 43], [25, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [21, 39], [29, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [1, 36], [32, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [26, 35], [33, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [17, 33], [35, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [26, 31], [37, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [5, 27], [41, 69], [9, 79], [0, 80], [0, 0]]
[[0, 0], [7, 25], [

In [ ]:
1+1

In [ ]:
print(SAVED)

In [ ]:
%%time
BIG_NUM=2
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6[:80], kv, [], k_2+k_6[80:], k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

COLORED 0
to check: 150 150
[[0, 0], [54, 79], [0, 80], [0, 0], [0, 0], [0, 0]]
[[0, 0], [31, 78], [1, 80], [0, 0], [0, 0], [0, 0]]
[[0, 0], [4, 77], [2, 80], [0, 0], [0, 0], [0, 0]]
[0, 945, 910] [183, 686]
[[0, 0], [58, 77], [2, 80], [0, 0], [0, 0], [0, 0]]
[[0, 0], [36, 76], [3, 80], [0, 0], [0, 0], [0, 0]]
[[0, 0], [15, 75], [4, 80], [0, 0], [0, 0], [0, 0]]
[[0, 0], [65, 75], [4, 80], [0, 0], [0, 0], [0, 0]]
[[0, 0], [46, 74], [5, 80], [0, 0], [0, 0], [0, 0]]
[[0, 0], [32, 73], [6, 80], [0, 0], [0, 0], [0, 0]]
[0, 945, 910] [287, 411]
[0, 945, 910] [287, 413]
[[0, 0], [23, 72], [7, 80], [0, 0], [0, 0], [0, 0]]
[0, 945, 910] [303, 430]
[0, 945, 910] [303, 427]
[0, 945, 910] [303, 444]
[[0, 0], [10, 71], [8, 80], [0, 0], [0, 0], [0, 0]]
[0, 945, 910] [311, 407]
[0, 945, 910] [311, 438]
[0, 945, 910] [311, 663]
[[0, 0], [1, 70], [9, 80], [0, 0], [0, 0], [0, 0]]
[0, 945, 910] [315, 317]
[[0, 0], [66, 70], [9, 80], [0, 0], [0, 0], [0, 0]]
[0, 945, 910] [315, 698]
[0, 945, 910] [317, 413

In [ ]:
%%time
BIG_NUM=1
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6[:80], kv, [], k_2+k_6[80:], k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [15]:
print(SAVED)

[([0, 945, 910], [183, 686]), ([0, 945, 910], [287, 411]), ([0, 945, 910], [287, 413]), ([0, 945, 910], [303, 430]), ([0, 945, 910], [303, 427]), ([0, 945, 910], [303, 444]), ([0, 945, 910], [311, 407]), ([0, 945, 910], [311, 438]), ([0, 945, 910], [311, 663]), ([0, 945, 910], [315, 317]), ([0, 945, 910], [315, 698]), ([0, 945, 910], [317, 413]), ([0, 945, 910], [317, 429]), ([0, 945, 910], [318, 423]), ([0, 945, 910], [318, 442]), ([0, 945, 910], [318, 444]), ([0, 945, 910], [318, 694]), ([0, 945, 910], [318, 698]), ([0, 945, 910], [399, 407]), ([0, 945, 910], [399, 427]), ([0, 945, 910], [399, 429]), ([0, 945, 910], [399, 663]), ([0, 945, 910], [399, 667]), ([0, 945, 910], [414, 407]), ([0, 945, 910], [414, 442]), ([0, 945, 910], [414, 444]), ([0, 945, 910], [414, 663]), ([0, 945, 910], [414, 669]), ([0, 945, 910], [414, 694]), ([0, 945, 910], [414, 700]), ([0, 945, 910], [430, 691]), ([0, 945, 910], [430, 441]), ([0, 945, 910], [430, 492]), ([0, 945, 910], [430, 407]), ([0, 945, 910

In [ ]:
1+1

In [17]:
len(SAVED)

327

In [24]:
def build_blocks_by_conf(conf):
    vrtx = conf[0] + conf[1]
    lss = len(vrtx)
    k22 = []
    k33 = []
    for i in range(lss):
        for j in range(i+1, lss):
            if dist(vrtx[i], vrtx[j]) == 6:
                k22.append((vrtx[i], vrtx[j]))
    for i in range(lss):
        for j in range(i+1, lss):
            for j1 in range(j+1, lss):
                if dist(vrtx[i], vrtx[j]) == 6 and dist(vrtx[i], vrtx[j1]) == 6 and dist(vrtx[j1], vrtx[j]) == 6:
                    k33.append((vrtx[i], vrtx[j], vrtx[j1]))
    return k22, k33

In [23]:
build_blocks_by_conf(SAVED[-1])

[0, 945, 910, 700]


([(0, 945), (0, 910), (0, 700), (945, 910)], [(0, 945, 910)])

In [29]:
%%time

#решаем вопрос о ровно 3 вершинах
SSAVED = []
for gr in SAVED:
    if len(gr[1]) == 3:
        k2_ss, k3_ss = build_blocks_by_conf(gr)
        grph = simple_cook_graph_no_K4(base_conf=gr[0], add_conf=gr[1], kl3s = k3_ss, k_2=k_2, k_4=k_4, k_6=[], n=n, k=k)
        brb = is_colorable(grph, 11, timer)
        if not brb:
            print(gr)
            SSAVED.append((gr[0], gr[1]))
    else:
        k2_ss, k3_ss = build_blocks_by_conf(gr)
        BIG_NUM = 3 - len(gr[1])
        bruteforce_nolimits_nok4_short(to_check=k_6[80:], base_conf=gr[0], add_conf=gr[1], k_2=k_2, k_4=k_4, k2s=k2_ss, kl3s=k2_ss, how_much=BIG_NUM, timer=20, n=10, k=6)

analysing 1-or-less-but-at-least-1 finished
analysing 1-or-less-but-at-least-1 finished
[[0, 0], [57, 70], [79, 80], [0, 0], [0, 0], [0, 0]]
analysing 1-or-less-but-at-least-1 finished
analysing 1-or-less-but-at-least-1 finished
analysing 1-or-less-but-at-least-1 finished



KeyboardInterrupt



In [30]:
len([([0, 945, 910], [159]), ([0, 945, 910], [175]), ([0, 945, 910], [183]), ([0, 945, 910], [187]), ([0, 945, 910], [189]), ([0, 945, 910], [190]), ([0, 945, 910], [287]), ([0, 945, 910], [303]), ([0, 945, 910], [311]), ([0, 945, 910], [315]), ([0, 945, 910], [317]), ([0, 945, 910], [318]), ([0, 945, 910], [399]), ([0, 945, 910], [414]), ([0, 945, 910], [430]), ([0, 945, 910], [435]), ([0, 945, 910], [437]), ([0, 945, 910], [441]), ([0, 945, 910], [455]), ([0, 945, 910], [459]), ([0, 945, 910], [461]), ([0, 945, 910], [467]), ([0, 945, 910], [469]), ([0, 945, 910], [470]), ([0, 945, 910], [473]), ([0, 945, 910], [474]), ([0, 945, 910], [476]), ([0, 945, 910], [483]), ([0, 945, 910], [485]), ([0, 945, 910], [486]), ([0, 945, 910], [489]), ([0, 945, 910], [490]), ([0, 945, 910], [492]), ([0, 945, 910], [498]), ([0, 945, 910], [500]), ([0, 945, 910], [504]), ([0, 945, 910], [543]), ([0, 945, 910], [559]), ([0, 945, 910], [567]), ([0, 945, 910], [571]), ([0, 945, 910], [573]), ([0, 945, 910], [574]), ([0, 945, 910], [655]), ([0, 945, 910], [670]), ([0, 945, 910], [686]), ([0, 945, 910], [691]), ([0, 945, 910], [693]), ([0, 945, 910], [697]), ([0, 945, 910], [711]), ([0, 945, 910], [715]), ([0, 945, 910], [717]), ([0, 945, 910], [723]), ([0, 945, 910], [725]), ([0, 945, 910], [726]), ([0, 945, 910], [729]), ([0, 945, 910], [730]), ([0, 945, 910], [732]), ([0, 945, 910], [739]), ([0, 945, 910], [741]), ([0, 945, 910], [742]), ([0, 945, 910], [745]), ([0, 945, 910], [746]), ([0, 945, 910], [407]), ([0, 945, 910], [411]), ([0, 945, 910], [413]), ([0, 945, 910], [423]), ([0, 945, 910], [427]), ([0, 945, 910], [429]), ([0, 945, 910], [438]), ([0, 945, 910], [442]), ([0, 945, 910], [444]), ([0, 945, 910], [663]), ([0, 945, 910], [667]), ([0, 945, 910], [669]), ([0, 945, 910], [679]), ([0, 945, 910], [683]), ([0, 945, 910], [685]), ([0, 945, 910], [694]), ([0, 945, 910], [698]), ([0, 945, 910], [700])])

80